In [1]:
from transformers import AutoTokenizer,AutoModelForTokenClassification, TrainingArguments, Trainer,pipeline
from datasets import load_dataset,load_metric,DatasetDict
from transformers import DataCollatorForTokenClassification
from tqdm import tqdm
import numpy as np
import os

2022-10-08 15:16:58.318804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-08 15:16:58.769200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-08 15:16:58.769289: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-08 15:16:58.856457: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-08 15:17:00.317222: W tensorflow/stream_executor/platform/de

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
raw_datasets = load_dataset("rjac/biobert-ner-diseases-dataset")

Using custom data configuration rjac--biobert-ner-diseases-dataset-dfa4ed497ebcd0a5
Found cached dataset parquet (/home/rjac/.cache/huggingface/datasets/rjac___parquet/rjac--biobert-ner-diseases-dataset-dfa4ed497ebcd0a5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
ner_feature = raw_datasets["train"].features["tags"]
label_names = ner_feature.feature.names

In [5]:
model_checkpoint = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

In [7]:
def align_labels_with_tokens(labels, word_ids):
    
    new_labels = []
    current_word = None
    
    for word_id in word_ids:
        
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
            
        elif word_id is None:
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [8]:
def tokenize_and_align_labels(batch):
    
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)
    all_labels = batch["tags"]
    
    new_labels = []
    
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [10]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/16 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Loading cached processed dataset at /home/rjac/.cache/huggingface/datasets/rjac___parquet/rjac--biobert-ner-diseases-dataset-dfa4ed497ebcd0a5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-bb804e59f7b54dc6.arrow


In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [12]:
metric = load_metric("seqeval")

/tmp/ipykernel_7325/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [13]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [14]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [15]:
model.config.num_labels

3

In [16]:
batch_size=24

In [17]:
args = TrainingArguments(
    "biobert-ner",
    evaluation_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit = 3,
    save_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=10,
    weight_decay=0.005,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2
)

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

/home/rjac/anaconda3/envs/mlops-healthcare-domain/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 15488
  Num Epochs = 10
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 6460
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("model")
tokenizer_final = AutoTokenizer.from_pretrained("model")
model_final = AutoModelForTokenClassification.from_pretrained("model")
model_final.push_to_hub("biobert-ner-diseases-model",use_auth_token=os.getenv("TOKEN_HF"))
tokenizer_final.push_to_hub("biobert-ner-diseases-model",use_auth_token=os.getenv("TOKEN_HF"))

### Inference

In [2]:
tokenizer_final = AutoTokenizer.from_pretrained("rjac/biobert-ner-diseases-model")
model_final = AutoModelForTokenClassification.from_pretrained("rjac/biobert-ner-diseases-model")
pipe = pipeline("token-classification",model=model_final,tokenizer=tokenizer_final,aggregation_strategy="simple")

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [17]:
pipe("it is propably that you have a type of brain tumor and high cold and fever")

[{'entity_group': 'Disease',
  'score': 0.99946904,
  'word': 'brain tumor',
  'start': 39,
  'end': 50},
 {'entity_group': 'Disease',
  'score': 0.7061189,
  'word': 'cold',
  'start': 60,
  'end': 64},
 {'entity_group': 'Disease',
  'score': 0.60857755,
  'word': 'fever',
  'start': 69,
  'end': 74}]